# PROBLEMA 4 - Equação Diferencial de Segunda Ordem

Resolver a Lei Fundamental da Dinâmica (LFD) para determinar a posição do objeto em queda vertical, submetido à força de arraste do ar e à força peso:

$$m\frac{d^2x}{dt^2}(t) = mg - \alpha v(t)$$

com condições iniciais: $x(t_0) = x_0$ e $\frac{dx}{dt}(t_0) = v_0$

Transformamos a EDO de segunda ordem em um sistema de EDOs de primeira ordem:
- $u_1 = x$ (posição)
- $u_2 = v = \frac{dx}{dt}$ (velocidade)

Sistema resultante:
$$\frac{du_1}{dt} = u_2$$
$$\frac{du_2}{dt} = g - \frac{\alpha}{m}u_2$$

Método das diferenças finitas:
$$u_{1,j+1} = u_{1,j} + u_{2,j} \delta t$$
$$u_{2,j+1} = u_{2,j} + f(x, u_{1,j}, u_{2,j}) \delta t$$

Solução analítica para comparação:
$$x(t) = \frac{mg}{\alpha}\left(t + \frac{m}{\alpha}(e^{-\frac{\alpha}{m}t} - 1)\right) - \frac{mv_0}{\alpha}(e^{-\frac{\alpha}{m}t} - 1) + x_0$$

In [ ]:
# ETAPA 1: Definir parâmetros
using Plots

m = 1.0       # massa (kg)
g = 9.8       # gravidade (m/s²)
alpha = 1.0   # coeficiente de arraste (N/(m/s))
t0 = 0.0      # tempo inicial (s)
t1 = 5.0      # tempo final (s)
x0 = 0.0      # posição inicial (m)
v0 = 0.0      # velocidade inicial (m/s)

In [ ]:
# ETAPA 2: Implementar método numérico (Euler para sistema de EDOs)
# Sistema: dx/dt = v,  dv/dt = g - (alpha/m)*v

function resolver_queda(n_pontos)
    dt = (t1 - t0) / (n_pontos - 1)
    t = zeros(n_pontos)
    x = zeros(n_pontos)
    v = zeros(n_pontos)
    
    # Condições iniciais
    t[1] = t0
    x[1] = x0
    v[1] = v0
    
    # Método de Euler
    for i in 1:(n_pontos-1)
        t[i+1] = t[i] + dt
        x[i+1] = x[i] + v[i] * dt
        v[i+1] = v[i] + (g - alpha/m * v[i]) * dt
    end
    
    return t, x, v
end

In [ ]:
# ETAPA 3: Calcular soluções analíticas
function x_analitico(t)
    exp_t = exp(-alpha * t / m)
    termo1 = (m*g/alpha) * (t + (m/alpha) * (exp_t - 1))
    termo2 = -(m*v0/alpha) * (exp_t - 1)
    return termo1 + termo2 + x0
end

function v_analitico(t)
    exp_t = exp(-alpha * t / m)
    return (m*g/alpha) * (1 - exp_t) + v0 * exp_t
end

# Calcular solução analítica
t_anal = range(t0, t1, length=1000)
x_anal = x_analitico.(t_anal)
v_anal = v_analitico.(t_anal)
v_limite = m * g / alpha  # Velocidade terminal

In [ ]:
# ETAPA 4: Plotar posição e velocidade vs tempo
cores = [:red, :green, :blue]
n_values = [50, 100, 500]

# Gráfico de Posição
p1 = plot(t_anal, x_anal, linewidth=2, color=:black, 
          label="Analítico", title="Posição vs Tempo", 
          xlabel="Tempo (s)", ylabel="Posição (m)")

for (i, n) in enumerate(n_values)
    t, x, v = resolver_queda(n)
    plot!(p1, t, x, line=:dash, color=cores[i], label="n=$n")
end

# Gráfico de Velocidade
p2 = plot(t_anal, v_anal, linewidth=2, color=:black, 
          label="Analítico", title="Velocidade vs Tempo",
          xlabel="Tempo (s)", ylabel="Velocidade (m/s)")

for (i, n) in enumerate(n_values)
    t, x, v = resolver_queda(n)
    plot!(p2, t, v, line=:dash, color=cores[i], label="n=$n")
end

hline!(p2, [v_limite], line=:dot, color=:purple, 
       label="v_limite = $(v_limite) m/s")

plot(p1, p2, layout=(1,2), size=(1200,500))

In [ ]:
# ETAPA 5: Espaço de fases (Velocidade vs Posição)
t_num, x_num, v_num = resolver_queda(500)

p = plot(x_anal, v_anal, linewidth=2, color=:black,
         label="Analítico", xlabel="Posição (m)", 
         ylabel="Velocidade (m/s)", 
         title="Espaço de Fases")

plot!(p, x_num, v_num, line=:dash, color=:red, label="Numérico (n=500)")
scatter!(p, [x0], [v0], markersize=8, color=:green, label="Início")
scatter!(p, [x_num[end]], [v_num[end]], markersize=8, color=:red, label="Fim")

plot(p, size=(800,600))

## Análise dos Resultados

### Observações Principais:

1. **Convergência para velocidade limite**: O objeto atinge aproximadamente a velocidade limite $v_{limite} = \frac{mg}{\alpha} = 9.8$ m/s

2. **Precisão do método numérico**: O método de Euler converge para a solução analítica à medida que aumentamos o número de pontos

3. **Comportamento físico**: 
   - Inicialmente, a aceleração é máxima (g = 9.8 m/s²)
   - À medida que a velocidade aumenta, a força de arraste cresce
   - Eventualmente, peso e arraste se equilibram (velocidade limite)

4. **Erro numérico**: O erro diminui com o aumento do número de pontos, mas o método de Euler é de primeira ordem, então a convergência é relativamente lenta

### Equações Implementadas:

- **Sistema de EDOs**:
  - $\frac{dx}{dt} = v$
  - $\frac{dv}{dt} = g - \frac{\alpha}{m}v$

- **Método de Euler**:
  - $x_{j+1} = x_j + v_j \Delta t$
  - $v_{j+1} = v_j + (g - \frac{\alpha}{m}v_j) \Delta t$

- **Solução analítica**:
  - $x(t) = \frac{mg}{\alpha}\left(t + \frac{m}{\alpha}(e^{-\frac{\alpha}{m}t} - 1)\right) - \frac{mv_0}{\alpha}(e^{-\frac{\alpha}{m}t} - 1) + x_0$